In [9]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import scipy.stats as ss
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
s0 = 49
mu = 0
k = 50
sigma = 0.2
q = 0
r = 0.05
T = 20 # weeks
numOfCall = 100000
timeBtwHedgeRebal = 1 # 단위 : 년

In [4]:
def d1(s,k,r,q,sigma,tau):
    return (np.log(s/k) + (r-q+(sigma**2)/2)*tau) / (sigma*np.sqrt(tau))

In [5]:
def d2(s,k,r,q,sigma,tau):
    return d1(s,k,r,q,sigma,tau) - sigma*np.sqrt(tau)

In [6]:
def delta(s,k,r,q,sigma,tau):
    return ss.norm.cdf(d1(s,k,r,q,sigma,tau))

In [7]:
def optionPrice(s,k,r,q,sigma,tau):
    return s*np.exp(-q*tau)*ss.norm.cdf(d1(s,k,r,q,sigma,tau)) - k*np.exp(-r*tau)*ss.norm.cdf(d2(s,k,r,q,sigma,tau))

In [19]:

def hedgingCost(s0,mu,k,r,q,sigma,T,numOfCall,timeBtwHedgeRebal):
    ### week ###
    weeksToYears = np.arange(0,T+1,timeBtwHedgeRebal)/52
    
    ### stock path ###
    total_steps = len(weeksToYears)
    dt = timeBtwHedgeRebal/52
    
    s = np.zeros(total_steps-1)
    s = np.insert(s, 0, s0)
    for i in range(total_steps-1):
        z = np.random.randn(1)
        s[i+1] = s[i] * np.exp((mu-0.5*sigma**2)*dt + np.sqrt(dt)*(sigma*z))
    
    #s = np.array([49.00, 48.125, 47.375, 50.25, 51.75, 53.125, 53.00, 51.875, 51.375, 53.00, 49.875, 48.50, 49.875, 50.375, 52.125, 51.875, 52.875, 54.875, 54.625, 55.875, 57.25]) ### Text book example
    #s = np.array([49.00, 49.75, 52.0, 50.00, 48.38, 48.25, 48.75, 49.63, 48.25, 48.25, 51.12, 51.50, 49.88, 49.88, 48.75, 47.50, 48.00, 46.25, 48.13, 46.63, 48.12]) ### Text book example
    
    ### delta path ###
    delta_path = np.zeros(total_steps)
    for i in range(total_steps):
        delta_path[i] = delta(s[i],k,r,q,sigma,weeksToYears[len(weeksToYears)-(i+1)])
                
    ### Shares purchage ###
    sharesPurchased = np.zeros(total_steps)
    for i in range(total_steps):
        if i == 0:
            sharesPurchased[i] = delta_path[i] * numOfCall
        else:
            sharesPurchased[i] = (delta_path[i] - delta_path[i-1]) * numOfCall
     
    ### Cost of shares purchased ###
    costOfSharesPurchased = s * sharesPurchased
        
    ### Interest Cost & Cumulative cost including interest ###
    interestCost = np.zeros(total_steps)
    cumCostInclInterest = np.zeros(total_steps)
    for i in range(total_steps):
        if i == 0:
            cumCostInclInterest[i] = costOfSharesPurchased[i]
            interestCost[i] = cumCostInclInterest[i] * r * dt
    
        else:
            cumCostInclInterest[i] = cumCostInclInterest[i-1] + costOfSharesPurchased[i] + interestCost[i-1]
            
            if i == total_steps-1:
                interestCost[i] = 0.0
            else:
                interestCost[i] = cumCostInclInterest[i] * r * dt
    
    hedgingCost = cumCostInclInterest[-1] - k*numOfCall

    return np.round(hedgingCost,2)

In [53]:
print(hedgingCost(s0,mu,k,r,q,sigma,T,numOfCall,timeBtwHedgeRebal))

-4740512.14


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


In [58]:

def hedgingCostSim(s0, mu, k, r, q, sigma, T, numOfCall, timeBtwHedgeRebal, sim_num):
    ### week ###
    weeksToYears = np.arange(0,T+1,timeBtwHedgeRebal)/52
    
    ### stock path ###
    total_steps = len(weeksToYears)
    dt = timeBtwHedgeRebal/52
    
    s = np.zeros((total_steps, sim_num))
    s[0,:] = s0
    for i in range(total_steps-1):
        z = np.random.randn(sim_num)
        s[i+1,:] = s[i,:] * np.exp((mu-0.5*sigma**2)*dt + np.sqrt(dt)*(sigma*z))
    #s = np.array([49.00, 48.125, 47.375, 50.25, 51.75, 53.125, 53.00, 51.875, 51.375, 53.00, 49.875, 48.50, 49.875, 50.375, 52.125, 51.875, 52.875, 54.875, 54.625, 55.875, 57.25]) ### Text book example
    #s = np.array([49.00, 49.75, 52.0, 50.00, 48.38, 48.25, 48.75, 49.63, 48.25, 48.25, 51.12, 51.50, 49.88, 49.88, 48.75, 47.50, 48.00, 46.25, 48.13, 46.63, 48.12]) ### Text book example
    
    ### delta path ###
    delta_path = np.zeros((total_steps, sim_num))
    for j in range(sim_num):
        for i in range(total_steps):
            delta_path[i,j] = delta(s[i,j],k,r,q,sigma,weeksToYears[len(weeksToYears)-(i+1)])
                
    ### Shares purchage ###
    sharesPurchased = np.zeros((total_steps, sim_num))
    for j in range(sim_num):
        for i in range(total_steps):
            if i == 0:
                sharesPurchased[i,j] = delta_path[i,j] * numOfCall
            else:
                sharesPurchased[i,j] = (delta_path[i,j] - delta_path[i-1,j]) * numOfCall
     
    ### Cost of shares purchased ###
    costOfSharesPurchased = s * sharesPurchased
        
    ### Interest Cost & Cumulative cost including interest ###
    interestCost = np.zeros((total_steps, sim_num))
    cumCostInclInterest = np.zeros((total_steps, sim_num))
    for j in range(sim_num):
        for i in range(total_steps):
            if i == 0:
                cumCostInclInterest[i,j] = costOfSharesPurchased[i,j]
                interestCost[i,j] = cumCostInclInterest[i,j] * r * dt

            else:
                cumCostInclInterest[i,j] = cumCostInclInterest[i-1,j] + costOfSharesPurchased[i,j] + interestCost[i-1,j]

                if i == total_steps-1:
                    interestCost[i,j] = 0.0
                else:
                    interestCost[i,j] = cumCostInclInterest[i,j] * r * dt
                    
    hedgingCost = cumCostInclInterest[-1,:] - k * numOfCall * (s[-1, :] > k)

    return np.round(hedgingCost,2)

In [61]:
sim_result = hedgingCostSim(s0,mu,k,r,q,sigma,T,numOfCall,timeBtwHedgeRebal, 10000)
sim_result
hedging_performance = sim_result.std() / (optionPrice(s0,k,r,q,sigma,T) * numOfCall)
sim_result.std()
hedging_performance

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  


array([146866.83, 246127.36, 262801.81, ..., 229480.33, 250672.22,
       258102.14])

46426.49784124948

0.014352213305405927